In [4]:
# region for 水印嵌入流程：
## 1. 计算0层O层的B*A矩阵（从PT文件中读取出B、A），记结果为O矩阵
## 2. uuid映射O矩阵
## 3. 12*1024矩阵转为张量
## 4. 图像水印算法
## 5. 张量转回12*1024矩阵
## 6. 预训练模型计算增量矩阵
## 7. BA矩阵分别与对应的增量阵相加
# endregion

In [5]:
# 第一步：计算0层O层的B*A矩阵
import torch
 
def calculate_O_matrix(lora_path):
    # 加载LoRA权重文件
    lora_data = torch.load(lora_path, map_location=torch.device('cpu'))
    
    # 获取A矩阵和B矩阵
    lora_A = lora_data['model.layers.0.self_attn.o_proj.lora_A.default.weight']
    lora_B = lora_data['model.layers.0.self_attn.o_proj.lora_B.default.weight']
    
    # 打印A矩阵和B矩阵的形状
    print("A矩阵形状:", lora_A.shape)
    print("B矩阵形状:", lora_B.shape)
    
    # 计算O矩阵 (B*A)
    O_matrix = torch.matmul(lora_B, lora_A)
    
    # 打印O矩阵的形状
    print("O矩阵形状:", O_matrix.shape)
    
    # 保存O矩阵到文件
    torch.save(O_matrix, 'O_matrix.pt')
    
    return O_matrix, lora_A, lora_B

lora_weights_path = "./lora_weights.pt"  # 替换为实际的LoRA权重文件路径
O_matrix, lora_A, lora_B = calculate_O_matrix(lora_weights_path)
print(O_matrix)

A矩阵形状: torch.Size([8, 2048])
B矩阵形状: torch.Size([2048, 8])
O矩阵形状: torch.Size([2048, 2048])
tensor([[ 4.4091e-05,  7.6450e-05, -1.3066e-04,  ...,  8.9990e-05,
         -6.9886e-05,  1.3933e-04],
        [ 9.5208e-05, -3.9194e-05, -2.5941e-04,  ..., -1.2387e-06,
         -1.1460e-04,  7.1963e-05],
        [-4.8367e-05, -3.9669e-05,  1.5975e-04,  ..., -2.8503e-05,
         -1.2687e-04,  2.1239e-04],
        ...,
        [-3.6304e-05,  8.5102e-05, -4.1491e-05,  ..., -6.3448e-05,
         -1.0415e-04,  1.3462e-04],
        [ 1.7444e-05, -4.5880e-05, -2.5448e-04,  ..., -4.9901e-05,
          4.4638e-06,  1.6786e-05],
        [-1.0263e-04, -4.6868e-05,  1.7759e-04,  ..., -7.3710e-05,
          1.3802e-05, -3.8382e-05]])


C:\Users\Administrator\AppData\Local\Temp\ipykernel_44320\703740492.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_data = torch.load(lora_path, map_location=torch.

In [6]:
# 第二步：uuid映射O矩阵
import numpy as np

def split_O_matrix(O_matrix):
    """将2048*2048的O矩阵分成四个1024*1024的子矩阵"""
    size = O_matrix.shape[0] // 2
    O1 = O_matrix[:size, :size]
    O2 = O_matrix[:size, size:]
    O3 = O_matrix[size:, :size]
    O4 = O_matrix[size:, size:]
    return O1, O2, O3, O4

def uuid_to_binary(uuid_str):
    """将UUID字符串转换为128位二进制，并分成4个32位"""
    # 移除所有破折号并转换为二进制
    uuid_int = int(uuid_str, 16)
    uuid_bin = format(uuid_int, '0128b')
    # 分成4个32位
    return [uuid_bin[i:i+32] for i in range(0, 128, 32)]

def adjust_indices(indices):
    """调整重复的索引值"""
    seen = set()
    result = []
    for idx in indices:
        while idx in seen:
            idx += 1
        seen.add(idx)
        result.append(idx)
    return result

def extract_vectors(O_part, uuid_part):
    """从O矩阵的子块提取三个向量"""
    # 提取前30位，分成3个10位
    indices = [int(uuid_part[i:i+10], 2) - 1 for i in range(0, 30, 10)]
    # 处理重复索引
    indices = adjust_indices(indices)
    
    # 获取后两位作为提取方式的标志
    extract_mode = int(uuid_part[30:], 2)  # 将最后两位转为整数(0-3)
    
    vectors = []
    for idx in indices:
        if extract_mode == 0:  # 按行取
            vector = O_part[idx % 1024, :].reshape(1, -1)
        elif extract_mode == 1:  # 按列取
            vector = O_part[:, idx % 1024].reshape(1, -1)
        elif extract_mode == 2:  # 按正对角线取
            # 从idx位置开始的对角线元素
            diag_vector = []
            for i in range(1024):
                row = (idx + i) % 1024
                col = (idx + i) % 1024
                diag_vector.append(O_part[row, col])
            vector = torch.tensor(diag_vector).reshape(1, -1)
        else:  # 按反对角线取
            # 从idx位置开始的反对角线元素
            anti_diag_vector = []
            for i in range(1024):
                row = (idx + i) % 1024
                col = (1023 - ((idx + i) % 1024))
                anti_diag_vector.append(O_part[row, col])
            vector = torch.tensor(anti_diag_vector).reshape(1, -1)
        
        vectors.append(vector)
    
    return vectors

# 其余代码保持不变
def process_matrix(O_matrix, uuid):
    # 1. 分割O矩阵
    O1, O2, O3, O4 = split_O_matrix(O_matrix)
    
    # 2. 将UUID转换为二进制并分割
    uuid_parts = uuid_to_binary(uuid)
    
    # 3. 从每个O矩阵部分提取向量
    O_parts = [O1, O2, O3, O4]
    all_vectors = []
    
    for O_part, uuid_part in zip(O_parts, uuid_parts):
        print("************************************************")
        print(type(O_part))
        print(O_part.shape)
        print(type(uuid_part))
        print(uuid_part)
        print("--------------------------------")
        vectors = extract_vectors(O_part, uuid_part)
        all_vectors.extend(vectors)
    
    # 4. 重组向量为x, y, z
    x = torch.cat([all_vectors[0], all_vectors[3], all_vectors[6], all_vectors[9]], dim=0)
    y = torch.cat([all_vectors[1], all_vectors[4], all_vectors[7], all_vectors[10]], dim=0)
    z = torch.cat([all_vectors[2], all_vectors[5], all_vectors[8], all_vectors[11]], dim=0)
    
    return x, y, z, all_vectors

# 使用示例
uuid = "c1d0f855d0f841d00c1d0000000000c1"
x, y, z, all_vectors = process_matrix(O_matrix, uuid)

# 打印结果形状
print("x形状:", x.shape)
print("y形状:", y.shape)
print("z形状:", z.shape)

# # 保存结果
# torch.save({
#     'x': x,
#     'y': y,
#     'z': z
# }, 'xyz_vectors.pt')
print(x)


************************************************
<class 'torch.Tensor'>
torch.Size([1024, 1024])
<class 'str'>
11000001110100001111100001010101
--------------------------------
************************************************
<class 'torch.Tensor'>
torch.Size([1024, 1024])
<class 'str'>
11010000111110000100000111010000
--------------------------------
************************************************
<class 'torch.Tensor'>
torch.Size([1024, 1024])
<class 'str'>
00001100000111010000000000000000
--------------------------------
************************************************
<class 'torch.Tensor'>
torch.Size([1024, 1024])
<class 'str'>
00000000000000000000000011000001
--------------------------------
x形状: torch.Size([4, 1024])
y形状: torch.Size([4, 1024])
z形状: torch.Size([4, 1024])
tensor([[ 5.9243e-05,  2.6908e-04, -6.0191e-05,  ...,  8.6427e-06,
         -1.4871e-04,  6.2912e-05],
        [ 5.7914e-05, -4.7265e-06,  1.4002e-04,  ..., -1.4388e-04,
          3.8581e-05, -1.3787e-04],
     

In [7]:
from prettytable import PrettyTable

x_subset = x[:, :512]
print(x_subset.shape)
table_x = PrettyTable()

# 添加列
table_x.field_names = [f"Col {i}" for i in range(x_subset.size(1))]
# 添加行
for row in x_subset:
    table_x.add_row(row.numpy())
# 打印表格
print(table_x)

print("--------------------------------")
y_subset = y[:, :512]
table_y = PrettyTable()

# 添加列
table_y.field_names = [f"Col {i}" for i in range(y_subset.size(1))]
# 添加行
for row in y_subset:
    table_y.add_row(row.numpy())
# 打印表格
print(table_y)
print("--------------------------------")

z_subset = z[:, :512]
table_z = PrettyTable()
# 添加列
table_z.field_names = [f"Col {i}" for i in range(z_subset.size(1))]
# 添加行
for row in z_subset:
    table_z.add_row(row.numpy())
# 打印表格
print(table_z)

torch.Size([4, 512])
+----------------+----------------+----------------+-----------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---------------+----------------+----------------+----------------+---------------+----------------+---------------+----------------+----------------+----------------+---------------+----------------+----------------+----------------+---------------+----------------+----------------+----------------+----------------+----------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---------------+----------------+---------------+---------------+----------------+----------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+---------------+----------------+----------------+----

In [8]:
# 第三步：12*1024矩阵转为张量,张量转为初始灰度图像（还有一个张量转回12*1024矩阵的函数）

## 先转为12*1024矩阵
combined = combined_matrix = torch.cat([x, y, z], dim=0)
print(combined.shape)
print(combined)

import torch
import numpy as np
from PIL import Image

def tensor_to_image(tensor, save_path='./watermark_image.png'):
    """
        将12*1024的矩阵=>归一化=>切割重组=>张量=>另存为48*256的图像
    """
    # 1. 归一化到[0,1]区间
    ## 归一化： 当前值 / 极差
    min_val = tensor.min()
    max_val = tensor.max()
    normalized = (tensor - min_val) / (max_val - min_val)
    
    # 2. 分割成4个12*256的块
    chunks = torch.chunk(normalized, 4, dim=1)  # 在列维度上分成4份
    
    # 3. 垂直堆叠成48*256
    stacked = torch.cat(chunks, dim=0)  # 48*256
    print(stacked)
    print("--------------------------------")
    # 4. 转换为图像格式（再次归一化到0-255区间）
    image_array = (stacked.numpy() * 255).astype(np.uint8)
    
    # 5. 创建并保存图像
    image = Image.fromarray(image_array, mode='L')  # L模式表示灰度图
    image.save(save_path)
    
    
    return image, stacked


# tensor是你的12*1024张量
image, stacked_tensor = tensor_to_image(combined, save_path='./original.png')



def image_to_tensor(stacked_tensor, original_tensor):
    """
    将48*256的stacked_tensor转回12*1024的原始tensor
    （因为嵌入水印是对48*256嵌入的，而且是归一化之后嵌入的，所以这里需要有一个逆归一化，把这种改动放回到B*A中，然后再利用预训练模型将B*A改动放回到B、A中）
    
    参数:
    stacked_tensor: 48*256的张量
    original_tensor: 原始的12*1024张量，用于获取原始值范围
    
    返回:
    restored_tensor: 恢复后的12*1024张量
    """
    # 1. 先将48*256转回12*1024的形状
    chunks = torch.chunk(stacked_tensor, 4, dim=0)  # 在行维度上分成4份，每份12*256
    normalized = torch.cat(chunks, dim=1)  # 水平拼接，变回12*1024
    
    # 2. 逆归一化，恢复到原始值范围
    min_val = original_tensor.min()
    max_val = original_tensor.max()
    restored_tensor = normalized * (max_val - min_val) + min_val
    
    return restored_tensor

restored_tensor = image_to_tensor(stacked_tensor, combined)
    
# 3. 验证误差
error = torch.abs(combined - restored_tensor)
max_error = torch.max(error)
mean_error = torch.mean(error)

print("***从归一化重组结果恢复到原始12*1024矩阵的误差***")
print("恢复出的12*1024矩阵：")
print(restored_tensor)
print(f"最大误差: {max_error:.10e}")
print(f"平均误差: {mean_error:.10e}")

torch.Size([12, 1024])
tensor([[ 5.9243e-05,  2.6908e-04, -6.0191e-05,  ...,  8.6427e-06,
         -1.4871e-04,  6.2912e-05],
        [ 5.7914e-05, -4.7265e-06,  1.4002e-04,  ..., -1.4388e-04,
          3.8581e-05, -1.3787e-04],
        [-1.5803e-04,  4.0164e-05,  2.6462e-04,  ...,  2.0340e-04,
          2.3261e-05, -2.1709e-05],
        ...,
        [ 2.9551e-05, -3.8048e-05,  6.7757e-05,  ..., -3.5014e-05,
         -7.2871e-05, -9.7401e-06],
        [-1.0263e-04, -4.6868e-05,  1.7759e-04,  ...,  3.2755e-05,
         -5.5963e-05, -9.2900e-05],
        [ 3.5599e-05,  7.1919e-05, -1.3474e-04,  ...,  7.1957e-05,
         -1.5052e-04, -3.7327e-06]])
tensor([[0.5343, 0.7709, 0.3996,  ..., 0.3826, 0.3327, 0.3751],
        [0.5328, 0.4622, 0.6254,  ..., 0.3757, 0.1624, 0.4188],
        [0.2893, 0.5128, 0.7659,  ..., 0.7453, 0.3091, 0.5619],
        ...,
        [0.3853, 0.4700, 0.4122,  ..., 0.4280, 0.3853, 0.4565],
        [0.2096, 0.5125, 0.6451,  ..., 0.5044, 0.4044, 0.3628],
        [0.3

<h2 align="left" style="font-size: 24px; color: #66ccff; background-color: #1c2833; padding: 10px 15px; border-left: 5px solid #3498db; margin: 15px 0;">嵌入比特序列的水印</h2>

<h3 align="left" style="font-size: 22px; color: #66ccff; background-color: #2c3e50; padding: 8px 15px; border-left: 4px solid #3498db; margin: 12px 0 12px 15px;">DCT+DWT</h3>

In [9]:
print(stacked_tensor)
print(stacked_tensor.shape)


tensor([[0.5343, 0.7709, 0.3996,  ..., 0.3826, 0.3327, 0.3751],
        [0.5328, 0.4622, 0.6254,  ..., 0.3757, 0.1624, 0.4188],
        [0.2893, 0.5128, 0.7659,  ..., 0.7453, 0.3091, 0.5619],
        ...,
        [0.3853, 0.4700, 0.4122,  ..., 0.4280, 0.3853, 0.4565],
        [0.2096, 0.5125, 0.6451,  ..., 0.5044, 0.4044, 0.3628],
        [0.3602, 0.6379, 0.3475,  ..., 0.5487, 0.2978, 0.4633]])
torch.Size([48, 256])


In [10]:
import numpy as np
from scipy.fft import dct, idct
import pywt
import torch  # 确保导入torch
import cv2

class HybridWatermark:
    def __init__(self, block_size=16, watermark_length=48):
        self.block_size = block_size
        self.watermark_length = watermark_length  # 水印长度（48bit）

    def dwt2(self, array):
        """使用DWT进行变换"""
        return pywt.dwt2(array, 'haar')

    def idwt2(self, coeffs):
        """使用IDWT进行逆变换"""
        return pywt.idwt2(coeffs, 'haar')
    
    def embed(self, stacked_tensor, watermark_bits):
        height, width = stacked_tensor.shape
        assert len(watermark_bits) == self.watermark_length, "Watermark length mismatch."
        
        # 使用 NumPy 的 copy() 方法来创建副本
        watermarked_tensor = stacked_tensor.copy()
        
        watermark_index = 0  # 水印位的索引
        count = 0
        position = []
        original_dct_coeffs = []  # 用于存储每个块的原始DCT系数

        # Step O: 准备图像块，分块大小为16x16，总共48个子块
        for h in range(0, height, self.block_size):
            for w in range(0, width, self.block_size):
                if watermark_index >= self.watermark_length:
                    break  # 水印已经嵌入完成

                # 获取当前块
                block = stacked_tensor[h:h+self.block_size, w:w+self.block_size]
                # tensor_temp = torch.from_numpy(block)[:4, :]
                # table_O = PrettyTable()
                # # 添加列
                # table_O.field_names = [f"Col {i}" for i in range(tensor_temp.size(1))]
                # # 添加行
                # for i, row in enumerate(tensor_temp):
                #     table_O.add_row(row.numpy())
                #     if i == 2:  # 例如在第 3 行后插入
                #         table_O.add_row(["..."] * tensor_temp.size(1))  # 添加省略号行
                # # 打印表格
                # print(table_O)


                position.append(((h, w), (h+self.block_size-1, w+self.block_size-1)))
                
                # Step 1: 执行DWT变换
                coeffs = self.dwt2(block)
                HH = coeffs[1][2]  # 获取HH子带（高频对角线）
                # tensor_temp = torch.from_numpy(HH)[[0, 1, 6, 7], :]
                # table_O = PrettyTable()
                # # 添加列
                # table_O.field_names = [f"Col {i}" for i in range(tensor_temp.size(1))]
                # # 添加行
                # for i, row in enumerate(tensor_temp):
                #     table_O.add_row(row.numpy())
                #     if i == 2:  # 例如在第 3 行后插入
                #         table_O.add_row(["..."] * tensor_temp.size(1))  # 添加省略号行
                # # 打印表格
                # print(table_O)


                # Step 2: 执行DCT变换
                C = dct(dct(HH.T, norm='ortho').T, norm='ortho')
                # print("HH子带区域DCT系数矩阵维度", C.shape)
                # tensor_temp = torch.from_numpy(C)[[0, 1, 6, 7], :]
                # table_O = PrettyTable()
                # # 添加列
                # table_O.field_names = [f"Col {i}" for i in range(tensor_temp.size(1))]
                # # 添加行
                # for i, row in enumerate(tensor_temp):
                #     table_O.add_row(row.numpy())
                #     if i == 2:  # 例如在第 3 行后插入
                #         table_O.add_row(["..."] * tensor_temp.size(1))  # 添加省略号行
                # # 打印表格
                # print(table_O)


                # 保存原始的DCT系数，用于后续提取计算
                original_dct_coeffs.append(C.copy())  # 保存原始系数

                # Step 3: 嵌入水印信息
                bit = watermark_bits[watermark_index]  # 当前要嵌入的水印比特
                x, y = 7, 7  # 高频位置
                if bit == 1:
                    C[x, y] += 0.1  # 增加量化步长
                else:
                    C[x, y] -= 0.1  # 减少量化步长

                # Step 4: DCT逆变换
                HH_modified = idct(idct(C.T, norm='ortho').T, norm='ortho')

                # Step 5: 逆DWT变换
                # 将修改后的HH替换到原始的coeffs中
                watermarked_block = self.idwt2((coeffs[0], (coeffs[1][0], coeffs[1][1], HH_modified)))  # 使用修改后的HH

                # 将水印嵌入到图像中，并确保转换为 NumPy 数组
                watermarked_tensor[h:h+self.block_size, w:w+self.block_size] = watermarked_block
                watermark_index += 1  # 处理下一个水印比特
                count += 1

        print(f"嵌入水印的块数量: {count}")
        print(position)
        return watermarked_tensor, original_dct_coeffs  # 返回水印图像和原始DCT系数


    def extract(self, watermarked_tensor, original_dct_coeffs):
        height, width = watermarked_tensor.shape
        extracted_bits = []

        watermark_index = 0  # 水印位的索引

        # Step O: 逐块分块并提取水印
        for h in range(0, height, self.block_size):
            for w in range(0, width, self.block_size):
                # 获取当前块
                block = watermarked_tensor[h:h+self.block_size, w:w+self.block_size]

                # Step 1: 执行DWT变换
                coeffs = self.dwt2(block)
                HH = coeffs[1][2]  # 获取HH子带（高频对角线）

                # Step 2: 执行DCT变换
                C = dct(dct(HH.T, norm='ortho').T, norm='ortho')

                # 获取原始DCT系数
                original_C = original_dct_coeffs[watermark_index]

                # Step 3: 计算变化量
                x, y = 7, 7  # 示例位置，选择DCT系数的位置（与嵌入时相同）
                diff = C[x, y] - original_C[x, y]  # 计算变化量

                # 根据DCT系数变化量判断水印
                if diff > 0:
                    extracted_bits.append(1)
                else:
                    extracted_bits.append(0)

                watermark_index += 1  # 处理下一个水印比特

        print(f"提取水印的总比特数量: {len(extracted_bits)}")
        return extracted_bits  # 返回提取的水印比特


def test_improved_watermark():
    # 假设 stacked_tensor 是一个示例的 PyTorch 张量（实际情况下你可以将其替换为真实的数据）
    stacked_tensor_np = stacked_tensor.numpy()  # 转换为 NumPy 数组
    # 水印信息，48位水印
    watermark_bits = [1, 0, 1, 1, 0, 1, 1, 0] * 6  # 48位水印
    # 初始化水印器
    watermarker1 = HybridWatermark(block_size=16, watermark_length=48)


    # 嵌入水印
    watermarked_tensor1, original_dct_coeffs = watermarker1.embed(stacked_tensor_np, watermark_bits)
    print("嵌入水印后的张量阵：")
    print(watermarked_tensor1.shape)

    # 提取水印
    extracted_bits1 = watermarker1.extract(watermarked_tensor1, original_dct_coeffs)
    print("提取水印结果:", extracted_bits1)

    # 计算MSE和准确率
    # 将 watermarked_tensor1 转换回 PyTorch 张量以进行比较
    watermarked_tensor1_torch = torch.tensor(watermarked_tensor1)

    mse1 = torch.mean((watermarked_tensor1_torch - stacked_tensor) ** 2).item()
    print(f"MSE: {mse1:.8f}")

    accuracy1 = sum(a == b for a, b in zip(watermark_bits, extracted_bits1)) / len(extracted_bits1)
    print(f"提取准确率: {accuracy1:.2%}")


    # 保存结果
    cv2.imwrite('watermarked_result1.png', (watermarked_tensor1_torch.numpy() * 255).astype(np.uint8))

    ## 测试误差
    print("************************************************")
    restored_tensor1 = image_to_tensor(watermarked_tensor1_torch, combined)
    print("逆向归一化后的矩阵维度", restored_tensor1.shape)
    tensor_temp = restored_tensor1[[0, 1,2,  9,10, 11], :]
    table_O = PrettyTable()
    # 添加列
    table_O.field_names = [f"Col {i}" for i in range(tensor_temp.size(1))]
    # 添加行
    for i, row in enumerate(tensor_temp):
        table_O.add_row(row.numpy())
        if i == 3:  # 例如在第 3 行后插入
            table_O.add_row(["..."] * tensor_temp.size(1))  # 添加省略号行
    # 打印表格
    print(table_O)


    # 3. 验证误差
    error1 = torch.abs(combined - restored_tensor1)
    max_error1 = torch.max(error1)
    mean_error1 = torch.mean(error1)

    print("***DCT+扩频序列嵌入水印恢复之后和原来12*1024矩阵的误差***")
    print("恢复出的12*1024矩阵：")
    print(restored_tensor1)
    print(f"最大误差: {max_error1:.10e}")
    print(f"平均误差: {mean_error1:.10e}")
    return restored_tensor1


if __name__ == "__main__":
    restored_tensor1 = test_improved_watermark()


嵌入水印的块数量: 48
[((0, 0), (15, 15)), ((0, 16), (15, 31)), ((0, 32), (15, 47)), ((0, 48), (15, 63)), ((0, 64), (15, 79)), ((0, 80), (15, 95)), ((0, 96), (15, 111)), ((0, 112), (15, 127)), ((0, 128), (15, 143)), ((0, 144), (15, 159)), ((0, 160), (15, 175)), ((0, 176), (15, 191)), ((0, 192), (15, 207)), ((0, 208), (15, 223)), ((0, 224), (15, 239)), ((0, 240), (15, 255)), ((16, 0), (31, 15)), ((16, 16), (31, 31)), ((16, 32), (31, 47)), ((16, 48), (31, 63)), ((16, 64), (31, 79)), ((16, 80), (31, 95)), ((16, 96), (31, 111)), ((16, 112), (31, 127)), ((16, 128), (31, 143)), ((16, 144), (31, 159)), ((16, 160), (31, 175)), ((16, 176), (31, 191)), ((16, 192), (31, 207)), ((16, 208), (31, 223)), ((16, 224), (31, 239)), ((16, 240), (31, 255)), ((32, 0), (47, 15)), ((32, 16), (47, 31)), ((32, 32), (47, 47)), ((32, 48), (47, 63)), ((32, 64), (47, 79)), ((32, 80), (47, 95)), ((32, 96), (47, 111)), ((32, 112), (47, 127)), ((32, 128), (47, 143)), ((32, 144), (47, 159)), ((32, 160), (47, 175)), ((32, 176), 

<h3 align="left" style="font-size: 22px; color: #66ccff; background-color: #2c3e50; padding: 8px 15px; border-left: 4px solid #3498db; margin: 12px 0 12px 15px;">恢复到B A中</h3>

In [11]:
## 首先将新的12*1024矩阵恢复到O矩阵（2048*2048）
"""
    需要放回去的矩阵：restored_tensor1
    放回到原始的O1,O2,O3,O4中
    然后重组O,记为O'，将O'作为后面模型的输入
    模型给出x,y
    分别将B更新为B+x，A更新为A+y
    这样就可以满足B'*A' == O'
"""
print(restored_tensor1.shape)


def reverse_mapping(O_matrix, uuid, C_new):
    """
    将扰动后的C矩阵(12*1024)中的向量放回原始位置
    
    参数:
    - O_matrix: 原始的2048*2048矩阵
    - uuid: 用于确定原始提取位置的uuid
    - C_new: 扰动后的12*1024矩阵，包含12个行向量
    
    返回:
    - 四个新的1024*1024矩阵 O1_new, O2_new, O3_new, O4_new
    """
    # 1. 分割原始O矩阵获取形状参考
    O1, O2, O3, O4 = split_O_matrix(O_matrix)
    O_parts = [O1.clone(), O2.clone(), O3.clone(), O4.clone()]  # 创建副本
    
    # 2. 解析uuid获取每个部分的提取模式和位置
    uuid_parts = uuid_to_binary(uuid)
    
    # 3. 从C_new中提取向量
    # x: 0-3行, y: 4-7行, z: 8-11行
    x_vectors = C_new[0:4]   # 4*1024
    y_vectors = C_new[4:8]   # 4*1024
    z_vectors = C_new[8:12]  # 4*1024
    
    # 4. 对每个O部分进行处理
    for part_idx, (O_part, uuid_part) in enumerate(zip(O_parts, uuid_parts)):
        # 获取该部分的三个索引
        indices = [int(uuid_part[i:i+10], 2) - 1 for i in range(0, 30, 10)]
        indices = adjust_indices(indices)  # 处理重复索引
        
        # 获取提取模式
        extract_mode = int(uuid_part[30:], 2)
        
        # 获取对应的新向量
        x_vec = x_vectors[part_idx].view(1, -1)
        y_vec = y_vectors[part_idx].view(1, -1)
        z_vec = z_vectors[part_idx].view(1, -1)
        vectors = [x_vec, y_vec, z_vec]
        
        # 将向量放回对应位置
        for idx, vec in zip(indices, vectors):
            idx = idx % 1024  # 确保索引在范围内
            if extract_mode == 0:  # 按行取
                O_parts[part_idx][idx, :] = vec.squeeze()
                
            elif extract_mode == 1:  # 按列取
                O_parts[part_idx][:, idx] = vec.squeeze()
                
            elif extract_mode == 2:  # 按正对角线取
                for i in range(1024):
                    row = (idx + i) % 1024
                    col = (idx + i) % 1024
                    O_parts[part_idx][row, col] = vec[0, i]
                    
            else:  # 按反对角线取
                for i in range(1024):
                    row = (idx + i) % 1024
                    col = (1023 - ((idx + i) % 1024))
                    O_parts[part_idx][row, col] = vec[0, i]
    
    return O_parts[0], O_parts[1], O_parts[2], O_parts[3]

O1_new, O2_new, O3_new, O4_new = reverse_mapping(O_matrix, uuid, restored_tensor1)
print(O1_new.shape)
print(O2_new.shape)
print(O3_new.shape)
print(O4_new.shape)


torch.Size([12, 1024])
torch.Size([1024, 1024])
torch.Size([1024, 1024])
torch.Size([1024, 1024])
torch.Size([1024, 1024])


In [12]:
print(restored_tensor1)
print(restored_tensor1.shape)

tensor([[ 5.9665e-05,  2.6865e-04, -6.1393e-05,  ...,  1.3763e-05,
         -1.4691e-04,  6.1114e-05],
        [ 5.7492e-05, -4.3045e-06,  1.4122e-04,  ..., -1.4900e-04,
          3.6783e-05, -1.3608e-04],
        [-1.5923e-04,  4.1366e-05,  2.6804e-04,  ...,  1.9736e-04,
          2.1140e-05, -1.9588e-05],
        ...,
        [ 2.7430e-05, -3.5927e-05,  7.3798e-05,  ..., -3.8435e-05,
         -7.4073e-05, -8.5387e-06],
        [-1.0442e-04, -4.5070e-05,  1.8271e-04,  ...,  3.1553e-05,
         -5.6385e-05, -9.2478e-05],
        [ 3.7397e-05,  7.0121e-05, -1.3986e-04,  ...,  7.3158e-05,
         -1.5010e-04, -4.1546e-06]])
torch.Size([12, 1024])


In [13]:
## 然后再以O'作为模型输入，生成x,y

import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import os
import torch.multiprocessing as mp  # 添加这行
import logging  # 添加这行

class ComplexMatrixSolver(nn.Module):
    def __init__(self):
        super(ComplexMatrixSolver, self).__init__()
        
        # 编码器
        self.encoder = nn.Sequential(
            nn.Linear(2048 * 16, 4096),
            nn.LayerNorm(4096),
            nn.LeakyReLU(0.2),
            nn.Linear(4096, 2048),
            nn.LayerNorm(2048),
            nn.LeakyReLU(0.2),
            nn.Linear(2048, 1024),
            nn.LayerNorm(1024),
            nn.LeakyReLU(0.2)
        )
        
        # 解码器X
        self.decoder_x = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.LayerNorm(2048),
            nn.LeakyReLU(0.2),
            nn.Linear(2048, 2048 * 8),
            nn.Tanh()
        )
        
        # 解码器Y
        self.decoder_y = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.LayerNorm(2048),
            nn.LeakyReLU(0.2),
            nn.Linear(2048, 8 * 2048),
            nn.Tanh()
        )

    def forward(self, A, B):
        batch_size = A.size(0)
        flat_A = A.view(batch_size, -1)
        flat_B = B.view(batch_size, -1)
        combined = torch.cat([flat_A, flat_B], dim=1)
        
        encoded = self.encoder(combined)
        x = self.decoder_x(encoded).view(batch_size, 2048, 8)
        y = self.decoder_y(encoded).view(batch_size, 8, 2048)
        
        return x, y

def load_model(model_path):
    """
    加载预训练模型
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ComplexMatrixSolver().to(device)
    
    # 加载模型权重
    state_dict = torch.load(model_path, map_location=device)
    # 移除"module."前缀（如果有的话）
    state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    
    model.eval()
    return model, device

def calculate_error(x, y, A, B, C):
    """
    计算误差矩阵和相对误差
    """
    # 计算预测
    BA = torch.bmm(B, A)
    By = torch.bmm(B, y)
    xA = torch.bmm(x, A)
    xy = torch.bmm(x, y)
    
    # 计算误差矩阵
    prediction = BA + By + xA + xy
    error_matrix = C - prediction
    
    # 计算相对误差
    relative_error = torch.norm(error_matrix) / torch.norm(C)
    
    return error_matrix, relative_error.item()

def infer(A, B, C, model_path='./models/checkpoint_2000.pth'):
    """
    使用预训练模型进行推理
    """
    # 加载模型
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ComplexMatrixSolver().to(device)
    
    # 加载检查点
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    # 确保输入是批次形式
    if len(A.shape) == 2:
        A = A.unsqueeze(0)
        B = B.unsqueeze(0)
        C = C.unsqueeze(0)
    
    # 移动数据到GPU
    A = A.to(device)
    B = B.to(device)
    C = C.to(device)
    
    # 推理
    with torch.no_grad():
        x, y = model(A, B)
        error_matrix, relative_error = calculate_error(x, y, A, B, C)
    
    # 如果输入不是批次，则移除批次维度
    if len(A.shape) == 3 and A.shape[0] == 1:
        x = x.squeeze(0)
        y = y.squeeze(0)
        error_matrix = error_matrix.squeeze(0)
    
    return x.cpu(), y.cpu(), error_matrix.cpu(), relative_error

def combine_O_matrix(O1, O2, O3, O4):
    """
    将四个1024*1024的矩阵重组为一个2048*2048的矩阵
    
    参数:
    - O1, O2, O3, O4: 四个1024*1024的矩阵
    
    返回:
    - O_new: 2048*2048的矩阵
    """
    # 先水平拼接
    top_half = torch.cat([O1, O2], dim=1)     # 1024 * 2048
    bottom_half = torch.cat([O3, O4], dim=1)   # 1024 * 2048
    
    # 再垂直拼接
    O_new = torch.cat([top_half, bottom_half], dim=0)  # 2048 * 2048
    
    return O_new

if __name__ == '__main__':
    
    O_new = combine_O_matrix(O1_new, O2_new, O3_new, O4_new)

    x, y, error_matrix, relative_error = infer(lora_A, lora_B, O_new)
    
    print(f"输入形状:")
    print(f"A shape: {lora_A.shape}")
    print(f"B shape: {lora_B.shape}")
    print(f"C shape: {O_new.shape}")
    
    print(f"\n输出形状:")
    print(f"x shape: {x.shape}")
    print(f"y shape: {y.shape}")
    print(f"Error matrix shape: {error_matrix.shape}")
    print(f"Relative error: {relative_error:.6f}")
    
    # 保存结果
    torch.save({
        'x': x,
        'y': y,
        'error_matrix': error_matrix,
        'relative_error': relative_error
    }, 'inference_results.pth')
    print(error_matrix)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_44320\3635541547.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=d

输入形状:
A shape: torch.Size([8, 2048])
B shape: torch.Size([2048, 8])
C shape: torch.Size([2048, 2048])

输出形状:
x shape: torch.Size([2048, 8])
y shape: torch.Size([8, 2048])
Error matrix shape: torch.Size([2048, 2048])
Relative error: 0.157830
tensor([[ 1.7117e-05, -6.6440e-06, -5.5619e-07,  ...,  9.0584e-07,
         -1.4627e-05, -1.3528e-05],
        [ 3.0348e-06, -1.6538e-05,  1.9397e-05,  ..., -6.6720e-06,
          7.1502e-06, -1.4037e-05],
        [-1.2182e-05,  3.3729e-05, -1.9839e-05,  ..., -4.4790e-06,
          1.0005e-06, -6.4749e-06],
        ...,
        [-2.4147e-05,  1.8856e-05,  2.7940e-05,  ...,  4.1624e-06,
          2.5830e-05,  1.0935e-06],
        [-1.0389e-05, -5.8897e-05,  2.1112e-05,  ..., -3.3732e-05,
         -4.4952e-05,  5.7381e-05],
        [-8.9039e-06,  3.3674e-05, -5.1292e-06,  ..., -2.0742e-05,
          5.9071e-06,  1.7889e-06]])


In [19]:
if __name__ == '__main__':
    # 假设 lora_A 和 lora_B 已经定义并计算
    # 先将 O_new 生成
    O_new = combine_O_matrix(O1_new, O2_new, O3_new, O4_new)

    # 保存 lora_A, lora_B 和 O_new
    torch.save(lora_A, 'lora_A.pt')
    torch.save(lora_B, 'lora_B.pt')
    torch.save(O_new, 'O_new.pt')

    # 动态读取 lora_A, lora_B 和 O_new
    lora_A_loaded = torch.load('lora_A.pt')
    lora_B_loaded = torch.load('lora_B.pt')
    O_new_loaded = torch.load('O_new.pt')

    # 使用加载的矩阵进行推理
    x, y, error_matrix, relative_error = infer(lora_A, lora_B, O_new)
    new_A = lora_A+y
    new_B = lora_B+x

    print(f"输入形状:")
    print(f"A shape: {lora_A_loaded.shape}")
    print(f"B shape: {lora_B_loaded.shape}")
    print(f"C shape: {O_new_loaded.shape}")
    print(f"new_A shape: {new_A.shape}")
    print(f"new_B shape: {new_B.shape}")
    torch.save(lora_A, 'lora_A_1.pt')
    torch.save(lora_B, 'lora_B_1.pt')
    print(f"\n输出形状:")
    print(f"x shape: {x.shape}")
    print(f"y shape: {y.shape}")
    print(f"Error matrix shape: {error_matrix.shape}")
    print(f"Relative error: {relative_error:.6f}")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_44320\894359270.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_A_loaded = torch.load('lora_A.pt')
C:\Users\Adm

输入形状:
A shape: torch.Size([8, 2048])
B shape: torch.Size([2048, 8])
C shape: torch.Size([2048, 2048])
new_A shape: torch.Size([8, 2048])
new_B shape: torch.Size([2048, 8])

输出形状:
x shape: torch.Size([2048, 8])
y shape: torch.Size([8, 2048])
Error matrix shape: torch.Size([2048, 2048])
Relative error: 0.157830


In [3]:
import torch
lora_data = torch.load('./O_new.pt', map_location=torch.device('cpu'))
print(lora_data.shape)

torch.Size([2048, 2048])


C:\Users\Administrator\AppData\Local\Temp\ipykernel_44320\3505226801.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_data = torch.load('./O_new.pt', map_location=to